<img src="https://investigacion.unirioja.es/img/ur-ddb7a50ad445db722a463e04d6373b1e.png" width="400" height="400" align="right" margin=80px;>
<p align="center">
<p><strong><u><span style="font-size: 20px;">Clasificador de Murci&eacute;lagos</span></u></strong></p>
<ul>
    <li>Ppip: Pipistrellus pipistrellus &nbsp;(Schreber, 1774) &nbsp;Murci&eacute;lago enano</li>
    <li>Ppyg: Pipistrellus pygmaeus &nbsp;(Leach, 1825) &nbsp;Murci&eacute;lago de Cabrera</li>
    <li>Pnat: Pipistrellus nathusii &nbsp;(Keyserling &amp; Blasius, 1839) &nbsp;Murci&eacute;lago de Nathusius</li>
    <li>Pkuh: Pipistrellus kuhlii &nbsp;(Kuhl, 1817) &nbsp;Murci&eacute;lago de borde claro</li>
    <li>Hsav: Hypsugo savii &nbsp;(Bonaparte, 1837) &nbsp;Murci&eacute;lago monta&ntilde;ero</li>
    <li>Msch: Miniopterus schreibersii &nbsp;(Kuhl, 1817) &nbsp;Murci&eacute;lago de cueva</li>
    <li>Plec: Plecotus</li>
    <li>Myot: Myotis</li>
    <li>Barb: Barbastella</li>
    <li>BACK: ruido de fondo(insectos u otro tipo)</li>
    <li>NOISE: ruido</li>
</ul>
<img src="https://www.researchgate.net/profile/Kate-Jones-7/publication/267212641/figure/fig2/AS:295588781412358@1447485265852/A-spectrogram-of-a-sequence-of-Pipistrellus-pipistrellus-echolocation-calls-Hanning.png" width="400" height="400" align="center">

In [1]:
import os, glob, random, time, sys, pickle
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa, librosa.display
import soundfile as sf
import shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
# import torch.optim as optim
import torch_optimizer as optim
from torchvision import transforms, models, datasets
# from torchlibrosa.stft import Spectrogram, LogmelFilterBank
# from torchlibrosa.augmentation import SpecAugmentation
from torchaudio.transforms import AmplitudeToDB
from transformers import get_linear_schedule_with_warmup

from datetime import datetime
from shutil import copyfile
from matplotlib.backends.backend_pdf import PdfPages
from tqdm.notebook import trange
# from tqdm import tqdm
from tqdm.notebook import tqdm
from functools import partial
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, ConfusionMatrixDisplay, confusion_matrix
from skimage.transform import resize
from PIL import Image, ImageOps
np.set_printoptions(suppress=True)
import progressbar
import timm
from timm.models.efficientnet import tf_efficientnet_b0_ns, tf_efficientnet_b1_ns, tf_efficientnet_b2_ns, tf_efficientnet_b3_ns, tf_efficientnet_b4_ns
from os import walk
from pandas import DataFrame
from fastai.vision.widgets import *
import zipfile
from ipyfilechooser import FileChooser
import itertools
from progress.bar import Bar

import warnings
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/home/rasieira/anaconda3/envs/murcielagos/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
class args:
    num_clases = 10,
    spec = {
        'n_fft' : 3072,
        'hop_length' : 768,
        'fmax' : 96000,
        'fmin' : 8000,
        'new_fmin':10000,
        'new_fmax':120000,
#         'new_fft':3072//2,
#         'new_hop_length':768//16,
        'new_hop_length':768//8,
        'new_fft':3072//2,
        'tramo': 1.5,
        'paso': 0.75,
        'batch':1
    }
device = ('cuda' if torch.cuda.is_available() else 'cpu')
SPECAUGMENT = False

In [3]:
class SedDataset:
    def __init__(self, df):
        self.df = df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):  
        name_files=[]
        images=[]
        intervalos=[]
        record = self.df.iloc[idx]
        name_file = record['full_path']
        y, sr = sf.read(name_file)
        len_crop = int(args.spec['tramo'] * sr)
        paso=int(args.spec['paso'] * sr)
        inf=0
        sup=len_crop
        i=0
        t=len(y)
        duracion=librosa.get_duration(filename=name_file)
        while len(y)<len_crop:
            y = np.concatenate([y, y])
        while sup<=len(y):
            aux=y[inf:sup]
            if len(aux)<len_crop:
                aux=y[len(y)-len_crop:len(y)]
            f_min = args.spec['fmin']
            f_max = args.spec['fmax']
            if sr==192000:
                n_fft= args.spec['n_fft'] // 2
                hop_length = args.spec['hop_length'] // 2
            if sr==256000:
                n_fft= args.spec['n_fft'] * 2 // 3
                hop_length = args.spec['hop_length'] * 2 // 3
            if sr==384000:
                n_fft= args.spec['n_fft']
                hop_length = args.spec['hop_length']

            frecuencias = np.arange(0, 1 + n_fft / 2) * sr / n_fft
            indices_freq = (frecuencias >= f_min) & (frecuencias <= f_max)
            img = np.abs(librosa.stft(aux, n_fft=n_fft, hop_length=hop_length, win_length=n_fft, window='hamming'))**2
            img = librosa.power_to_db(img, ref=np.max)
            img = img[indices_freq,:]
            img = torch.from_numpy(img)
            images.append(img)
            name_files.append(name_file)
            intervalos.append([inf/sr,sup/sr])
            inf=inf+paso
            sup=sup+paso
            i=i+1
        return {"images" : images,'name_files' : name_files,'intervalos':intervalos}

In [4]:
class SedDatasetNoInterval:
    def __init__(self, df, audio_transform=None, modo="train"):
        self.audio_transform = audio_transform
        self.modo = modo
        self.df = df

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        record = self.df.iloc[idx]
        name_file = record['full_path']
        y, sr = sf.read(name_file)
        len_crop = int(args.spec['tramo'] * sr)
        
        # Agumentation in audio
        if self.audio_transform:
            y = self.audio_transform(samples=y, sample_rate=sr)
        
        f_min = args.spec['new_fmin']
        f_max = args.spec['new_fmax']

        # Frecuencias y hop_length proporcionales a la sr
        if sr==192000:
            n_fft= args.spec['new_fft'] // 2
            hop_length = args.spec['new_hop_length'] // 2
        if sr==256000:
            n_fft= args.spec['new_fft'] * 2 // 3
            hop_length = args.spec['new_hop_length'] * 2 // 3
        if sr==384000:
            n_fft= args.spec['new_fft']
            hop_length = args.spec['new_hop_length']
        
        # Banda de frecuencias a filtrar
        frecuencias = np.arange(0, 1 + n_fft / 2) * sr / n_fft
        indices_freq = (frecuencias >= f_min) & (frecuencias <= f_max)
        
        # Reduce el paso a la mitad si sr==192000
        img = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length, win_length=n_fft, window='hamming'))**2
#         img = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length, win_length=n_fft, window=('kaiser', 0.0)))**2
        img = librosa.power_to_db(img, ref=np.max)
        img = img[indices_freq,:]
        
        # Specaugment
        img = torch.from_numpy(img).unsqueeze(0)
        if SPECAUGMENT:
            img = spec_augment(img, time_warping_para=1, 
                               time_masking_para=36,  time_mask_num=2,
                               frequency_masking_para=36, frequency_mask_num=2)
        img = transforms.Normalize(mean=[0.485], std=[0.229])(img)
        label = np.zeros(args.num_clases)
#         if self.modo=='train' or self.modo=='valid':
#             label[record['label_num']] = 1.0
        
        return {"image" : img, "target" : label, 'filename' : record['full_path'],'sr':sr,'new_fft':n_fft,'new_hop_length':hop_length}

In [5]:
def Model(arquitectura,clases):
    model = timm.create_model(arquitectura, pretrained=False, num_classes=clases, in_chans=1)
    return model

In [6]:
out_pl = widgets.Output()
out_pl2 = widgets.Output()
out_pl3 = widgets.Output()
out_pl4 = widgets.Output()

In [7]:
def on_click_classify(change):
    torch.cuda.empty_cache()
    paths=[]
    array = ['Ppip', 'Pnat-Pkuh','Ppyg-Msch','Hsav']
    now = datetime.now()
    date_time = now.strftime("%Y%m%d%H%M")
    for (dirpath, dirnames, filenames) in walk(fc.selected_path):
            for t in filenames:
                if t.endswith('.wav'):
                    completo=fc.selected_path+'/'+t
                    paths.append(str(completo))
    lista_modelos=[]
    aux=fc1.selected_path+'/SPECIES/'
    i=0
    for (folder, subfolder, file) in os.walk(aux):
            for filename in file:
                if filename.endswith('.bin'):
                    lista_modelos.append(folder+'/'+filename)
    df = DataFrame (paths,columns=['full_path'])
    inferenceset = SedDataset(df = df)
    inference_loader = DataLoader(inferenceset, batch_size=1, shuffle=False, drop_last=False, num_workers=2)
    diccionario={}
    etiquetas_ord = ['BACK', 'Barb', 'Hsav', 'Myot', 'NOISE', 'Nyct', 'Plec', 'Pnat-Pkuh', 'Ppip','Ppyg-Msch','Rhin']
#     etiquetas_ord=etiquetas_ord.upper()
    y_intervalos=[]
    y_name_files=[]
    new_y_pred=[]
    new_name_files=[]
    new_intervalos=[]
    arquitecturas=[]
    output_path=fc2.selected_path
    if output_path is None:
        output_path="/home/rasieira/Escritorio/"
    for b in lista_modelos:
        arquitectura=(b.split('/')[-2]).lower()
        if arquitectura=='resnext50':
            arquitectura='resnext50_32x4d'
        arquitecturas.append(arquitectura)
    numero_modelos=len(lista_modelos)
    for b in arquitecturas:
        diccionario[b]=[]
    for b in lista_modelos:
        arquitectura=(b.split('/')[-2]).lower()
        if arquitectura=='resnext50':
            arquitectura='resnext50_32x4d'
        model=Model(arquitectura,len(etiquetas_ord))
        model.load_state_dict(torch.load(b))
        diccionario[arquitectura].append(model)
    final_arquitecturas=set(arquitecturas)
    resultados=[]
    inferiores=[]
    superiores=[]
    nombres=[]
    modelos=[]
    with out_pl:
        with torch.no_grad():
            outer=tqdm(inference_loader)
            preds_df_max_sample=[]
            model_sample=[]
            numero_muestras=len(outer)
            #Recorremos cada audio
            for sample in outer:
                input=torch.stack(sample['images']).float().to(device)
                preds_df_max_batch=[]
                #Recorremos el batch de segmentos de un audio
                for i in trange(0,len(input),args.spec['batch'],leave=False):
                    pos_fin=args.spec['batch']+i
                    if pos_fin>len(input):
                        pos_fin=len(input)
                    img_batch=input[i:pos_fin]
                    model_batch=[]
                    preds_df_max_arquitectura=[]
                    #Recorremos los modelos
                    for a in diccionario:
                        pred_list=[]
                        models_arquitectura=[]
                        model_list=[]
                        #recorremos los folds de cada modelo
                        for p in diccionario[a]:
                            model=p
                            model.to(device).eval()
                            output = model(img_batch.float().to(device))
                            y_pred=torch.sigmoid(output).detach().cpu().numpy()
                            pred_list.append(y_pred)
                            del model
                            torch.cuda.empty_cache()
                        y_pred=np.stack(pred_list)
                        y_pred=y_pred.max(axis=0)
                        preds_df_max_arquitectura.append(y_pred)
                        modelos.append([a]*len(np.arange(i,pos_fin)))
                        inferiores.append([sample['intervalos'][k][0].cpu().numpy()[0] for k in np.arange(i,pos_fin)])
                        superiores.append([sample['intervalos'][k][1].cpu().numpy()[0] for k in np.arange(i,pos_fin)])
                        nombres.append([sample['name_files'][k][0] for k in np.arange(i,pos_fin)])    
                    preds_df_max_batch.append(np.vstack(preds_df_max_arquitectura))
                outer.update(1)
                resultados.append(np.vstack(preds_df_max_batch))
            outer.close()
    resultados=np.vstack(resultados)
    df_preds = pd.DataFrame(resultados,columns=etiquetas_ord)
    vector=df_preds.idxmax(axis=1)
    df_preds['Prediction']=np.array(vector)
    df_preds['name_file']=np.concatenate(nombres)
    df_preds['inf']=np.concatenate(inferiores)
    df_preds['sup']=np.concatenate(superiores)
    df_preds['model']=np.concatenate(modelos)
    df_preds=df_preds[['name_file','inf','sup','model']+etiquetas_ord+['Prediction']]
    categorias=['5/','4/','3/','2/','1/']
    if not os.path.exists(str(output_path)):
        os.makedirs(str(output_path))
    if not os.path.exists(str(output_path)+'/MULTIPLES/'):
        os.makedirs(str(output_path)+'/MULTIPLES/')
    if not os.path.exists(str(output_path)+'/SIMPLES/'):
        os.makedirs(str(output_path)+'/SIMPLES/')
    
    if not os.path.exists(str(output_path)+'/MULTIPLES/GRUPOS_FONICOS/'):
        os.makedirs(str(output_path)+'/MULTIPLES/GRUPOS_FONICOS/')
    if not os.path.exists(str(output_path)+'/SIMPLES/GRUPOS_FONICOS/'):
        os.makedirs(str(output_path)+'/SIMPLES/GRUPOS_FONICOS/')
    aux=etiquetas_ord.copy()
    for a in etiquetas_ord:
            if not os.path.exists(str(output_path)+'/MULTIPLES/GRUPOS_FONICOS/PPIT/'+a+'/') and a in array:
                os.makedirs(str(output_path)+'/MULTIPLES/GRUPOS_FONICOS/PPIT/'+a+'/')
            if not os.path.exists(str(output_path)+'/MULTIPLES/GRUPOS_FONICOS/'+a+'/') and a not in array:
                os.makedirs(str(output_path)+'/MULTIPLES/GRUPOS_FONICOS/'+a+'/')
            if not os.path.exists(str(output_path)+'/SIMPLES/GRUPOS_FONICOS/PPIT/'+a+'/') and a in array:
                os.makedirs(str(output_path)+'/SIMPLES/GRUPOS_FONICOS/PPIT/'+a+'/')
            if not os.path.exists(str(output_path)+'/SIMPLES/GRUPOS_FONICOS/'+a+'/') and a not in array:
                os.makedirs(str(output_path)+'/SIMPLES/GRUPOS_FONICOS/'+a+'/')
            for t in categorias:
                if not os.path.exists(str(output_path)+'/MULTIPLES/GRUPOS_FONICOS/PPIT/'+a+'/'+t+'/') and a in array:
                    os.makedirs(str(output_path)+'/MULTIPLES/GRUPOS_FONICOS/PPIT/'+a+'/'+t+'/')
                if not os.path.exists(str(output_path)+'/MULTIPLES/GRUPOS_FONICOS/'+a+'/'+t+'/') and a not in array:
                    os.makedirs(str(output_path)+'/MULTIPLES/GRUPOS_FONICOS/'+a+'/'+t+'/')
                if not os.path.exists(str(output_path)+'/SIMPLES/GRUPOS_FONICOS/PPIT/'+a+'/'+t+'/') and a in array:
                    os.makedirs(str(output_path)+'/SIMPLES/GRUPOS_FONICOS/PPIT/'+a+'/'+t+'/')
                if not os.path.exists(str(output_path)+'/SIMPLES/GRUPOS_FONICOS/'+a+'/'+t+'/') and a not in array:
                    os.makedirs(str(output_path)+'/SIMPLES/GRUPOS_FONICOS/'+a+'/'+t+'/') 
    umbral_murcielagos=umbral_groups.value
    porcentaje_aparicion=aparicion_species.value
    path_csv=str(output_path)+'/'+str(date_time)+'resultados.csv'
    aux_df_preds=df_preds.copy()
    aux_df_preds['name_file']=aux_df_preds['name_file'].str.split('/').str[-1]
    aux_df_preds.to_csv(path_csv,index=False)
    mat=df_preds.copy()
    mat2=mat.copy()
    diccionario={}
    for a in etiquetas_ord:
        mat2[a]=(mat2[a]>umbral_murcielagos)
        mat3=mat2.groupby(['name_file','model'],as_index=False)[[a]].mean()
        mat3[a]=(mat3[a]>porcentaje_aparicion)
        mat4=mat3.groupby(['name_file'],as_index=False)[[a]].sum()
        diccionario[a]=np.stack(mat4[a].values)
    final = pd.DataFrame.from_dict(diccionario)
    final['name_file']=mat4['name_file']
    a=final[etiquetas_ord]
    new = a.eq(a.max(axis=1), axis=0)
    o = new.mul(new.columns.to_series()).apply(','.join, axis=1).str.strip(',')
    t=[]
    for a in o:
        lista=a.replace(',',' ').split(' ')
        t.append([item for item in lista if item])
    final['Prediction']=np.array(t)
    for index,row in final.iterrows():
        original=row['name_file']
        name_file=original.split('/')[-1]
        if len(row['Prediction'])!=1:
            for a in row['Prediction']:
                if str(a) in array:
                    target=str(output_path)+'/MULTIPLES/GRUPOS_FONICOS/PPIT/'+str(a)+'/'+str(row[a])+'/'+name_file
                else:
                    target=str(output_path)+'/MULTIPLES/GRUPOS_FONICOS/'+str(a)+'/'+str(row[a])+'/'+name_file
                if not(os.path.exists(target)):
                    copyfile(original,target)
        else:
                if str(row['Prediction'][0]) in array:
                    target=str(output_path)+'/SIMPLES/GRUPOS_FONICOS/PPIT/'+str(row['Prediction'][0])+'/'+str(row[row['Prediction'][0]])+'/'+name_file
                else:
                    target=str(output_path)+'/SIMPLES/GRUPOS_FONICOS/'+str(row['Prediction'][0])+'/'+str(row[row['Prediction'][0]])+'/'+name_file
                if not(os.path.exists(target)):
                    copyfile(original,target)
        if os.path.exists(original):
            os.remove(original)
    final['Prediction']=final['Prediction'].astype(str)
    final['Prediction']=final['Prediction'].str.replace("[","").str.replace("]","").str.replace('"','').str.replace("'","")
    final['name_file']=final['name_file'].str.split('/').str[-1]
    final.to_csv(str(output_path)+'/'+str(date_time)+'votesbyfile.csv',index=False)
    with out_pl:   
        display(final)
        display(aux_df_preds)

In [8]:
def on_click_classify_2(change):
    torch.cuda.empty_cache()
    paths=[]
    now = datetime.now()
    date_time = now.strftime("%Y%m%d%H%M")
    for (dirpath, dirnames, filenames) in walk(fc3.selected_path):
            for t in filenames:
                if t.endswith('.wav'):
                    completo=fc3.selected_path+'/'+t
                    paths.append(str(completo))
    lista_modelos=[]
    aux=fc4.selected_path+'/NOISE/'
    output_path=fc5.selected_path
    if output_path is None:
        output_path="/home/rasieira/Escritorio/"
    i=0
    for (folder, subfolder, file) in os.walk(aux):
            for filename in file:
                if filename.endswith('.bin'):
                    lista_modelos.append(folder+'/'+filename)
    df = DataFrame (paths,columns=['full_path'])
    inferenceset = SedDataset(df = df)
    inference_loader = DataLoader(inferenceset, batch_size=1, shuffle=False, drop_last=False, num_workers=2)
    diccionario={}
    etiquetas_ord = ['BATS', 'NOISE']
    y_intervalos=[]
    y_name_files=[]
    new_y_pred=[]
    new_name_files=[]
    new_intervalos=[]
    arquitecturas=[]
    for b in lista_modelos:
        arquitectura=(b.split('/')[-2]).lower()
        if arquitectura=='resnext50':
            arquitectura='resnext50_32x4d'
        arquitecturas.append(arquitectura)
    numero_modelos=len(lista_modelos)
    for b in arquitecturas:
        diccionario[b]=[]
    for b in lista_modelos:
        arquitectura=(b.split('/')[-2]).lower()
        if arquitectura=='resnext50':
            arquitectura='resnext50_32x4d'
        model=Model(arquitectura,len(etiquetas_ord))
        model.load_state_dict(torch.load(b))
        diccionario[arquitectura].append(model)
    final_arquitecturas=set(arquitecturas)
    resultados=[]
    inferiores=[]
    superiores=[]
    nombres=[]
    modelos=[]
    with out_pl2:
        with torch.no_grad():
            outer=tqdm(inference_loader)
            preds_df_max_sample=[]
            model_sample=[]
            numero_muestras=len(outer)
            #Recorremos cada audio
            for sample in outer:
                input=torch.stack(sample['images']).float().to(device)
                preds_df_max_batch=[]
                #Recorremos el batch de segmentos de un audio
                for i in trange(0,len(input),args.spec['batch'],leave=False):
                    pos_fin=args.spec['batch']+i
                    if pos_fin>len(input):
                        pos_fin=len(input)
                    img_batch=input[i:pos_fin]
                    model_batch=[]
                    preds_df_max_arquitectura=[]
                    #Recorremos los modelos
                    for a in diccionario:
                        pred_list=[]
                        models_arquitectura=[]
                        model_list=[]
                        #recorremos los folds de cada modelo
                        for p in diccionario[a]:
                            model=p
                            model.to(device).eval()
                            output = model(img_batch.float().to(device))
                            y_pred=torch.sigmoid(output).detach().cpu().numpy()
                            pred_list.append(y_pred)
                            del model
                            torch.cuda.empty_cache()
                        y_pred=np.stack(pred_list)
                        y_pred=y_pred.max(axis=0)
                        preds_df_max_arquitectura.append(y_pred)
                        modelos.append([a]*len(np.arange(i,pos_fin)))
                        inferiores.append([sample['intervalos'][k][0].cpu().numpy()[0] for k in np.arange(i,pos_fin)])
                        superiores.append([sample['intervalos'][k][1].cpu().numpy()[0] for k in np.arange(i,pos_fin)])
                        nombres.append([sample['name_files'][k][0] for k in np.arange(i,pos_fin)])    
                    preds_df_max_batch.append(np.vstack(preds_df_max_arquitectura))
                outer.update(1)
                resultados.append(np.vstack(preds_df_max_batch))
            outer.close()
    resultados=np.vstack(resultados)
    df_preds = pd.DataFrame(resultados,columns=etiquetas_ord)
    vector=df_preds.idxmax(axis=1)
    df_preds['Prediction']=np.array(vector)
    df_preds['inf']=np.concatenate(inferiores)
    df_preds['sup']=np.concatenate(superiores)
    df_preds['model']=np.concatenate(modelos)
    df_preds['name_file']=np.concatenate(nombres)
    df_preds=df_preds[['name_file','inf','sup','model']+etiquetas_ord+['Prediction']]
    if not os.path.exists(str(output_path)+'/OUTPUT/'):
        os.makedirs(str(output_path)+'/OUTPUT/')
    if not os.path.exists(str(output_path)+'/OUTPUT/NOISE/'):
        os.makedirs(str(output_path)+'/OUTPUT/NOISE/')
    if not os.path.exists(str(output_path)+'/OUTPUT/BATS/'):
        os.makedirs(str(output_path)+'/OUTPUT/BATS/')
    path_csv=str(output_path)+'/OUTPUT/'+str(date_time)+'resultados.csv'
    aux_df_preds=df_preds.copy()
    aux_df_preds['name_file']=aux_df_preds['name_file'].str.split('/').str[-1]
    aux_df_preds.to_csv(path_csv,index=False)
    umbral_murcielagos=float(umbral_bats.value)
    umbral_noise=float(umbral_ruido.value)
    porcentaje_aparicion=float(aparicion_noise.value)
    mat=df_preds.copy()
    mat2=mat.copy()
    mat2['BATS']=(mat2['BATS']>umbral_murcielagos)
    mat2['NOISE']=(mat2['NOISE']>umbral_noise)
    aux_mat_2=mat2.copy()
    aux_mat_2['name_file']=aux_mat_2['name_file'].str.split('/').str[-1]
    aux_mat_2.to_csv(str(output_path)+'/OUTPUT/'+'/'+str(date_time)+'resultados_binary.csv',index=False)
    mat3=mat2.groupby(['name_file','model'],as_index=False)[['BATS','NOISE']].mean()
    mat3['BATS']=(mat3['BATS']>porcentaje_aparicion)
    mat3['NOISE']=(mat3['NOISE']>porcentaje_aparicion)
    aux_mat_3=mat3.copy()
    aux_mat_3['name_file']=aux_mat_3['name_file'].str.split('/').str[-1]
    aux_mat_3.to_csv(str(output_path)+'/OUTPUT/'+'/'+str(date_time)+'resultados_votebymodel.csv',index=False)
    mat4=mat3.groupby(['name_file'],as_index=False)[['BATS','NOISE']].sum()
    aux_mat_4=mat4.copy()
    aux_mat_4['name_file']=aux_mat_4['name_file'].str.split('/').str[-1] 
    aux_mat_4.to_csv(str(output_path)+'/OUTPUT/'+'/'+str(date_time)+'resultados_votes.csv',index=False)
    maxValueIndex = mat4[['NOISE','BATS']].idxmax(axis=1)
    mat4['Prediction']=np.array(maxValueIndex)
    for index,row in mat4.iterrows():
        original=row['name_file']
        name_file=original.split('/')[-1]
        target=str(output_path)+'/OUTPUT/'+str(row['Prediction'])+'/'+name_file
        if not(os.path.exists(target)):
            copyfile(original,target)
    with out_pl2:
        display(aux_df_preds)
        display(aux_mat_4)

In [9]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [10]:
def on_click_classify_3(change):
    output_path=fc7.selected_path
    if output_path is None:
        output_path="/home/rasieira/Escritorio/"
    SPECAUGMENT = False
    now = datetime.now()
    date_time = now.strftime("%Y%m%d%H%M")
    with out_pl3:
        seed_everything(1234)
        paths=[]
        for root, dirs, files in os.walk(fc6.selected_path):
            for file in files:
                if file.endswith(".wav"):
                     paths.append(os.path.join(root, file))
        df = DataFrame (paths,columns=['full_path'])
        display(df)
        validset = SedDatasetNoInterval(df = df, audio_transform=None, modo="valid")
        valid_loader = DataLoader(validset, batch_size=1, shuffle=False, drop_last=False, num_workers=0)
        for i, entrada in enumerate(tqdm(valid_loader)):
                fig, ax = plt.subplots()
                fig.set_figheight(10)
                fig.set_figwidth(20)
                arr=entrada['image'].cpu().numpy()[0,0,:,:]
#                 newArr=np.where(abs(arr)>100,0,100)
                img = librosa.display.specshow(arr,
                                               fmin=args.spec['new_fmin'],
                                               cmap='gray_r',
                                               fmax=args.spec['new_fmax'],
                                               hop_length=entrada['new_hop_length'].cpu().numpy(),
                                               sr=entrada['sr'].cpu().numpy(),
                                               x_axis='time', 
                                               y_axis=None, 
                                               ax=ax)
                freq_labels = np.linspace(args.spec['new_fmin'], args.spec['new_fmax'], 10).astype(int)
                freq_labels_pos = np.linspace(0,entrada['image'].cpu().numpy()[0,0,:,:].shape[0],len(freq_labels)).astype(int)
                plt.yticks(freq_labels_pos, freq_labels);
                fig.colorbar(img, ax=ax, format="%+2.f dB");
                nombre=entrada['filename'][0].split('/')[-1].split('.')[0]
                plt.title(nombre)
                plt.savefig(entrada['filename'][0].split('.')[0]+'.png', bbox_inches='tight');


In [11]:
fc = FileChooser('../')

# # Print the selected path, filename, or both
# if fc.selected_path is not None:
#     print(fc.selected_path)
# if fc.selected_filename is not None:
#     print(fc.selected_filename)
# if fc.selected is not None:
#     print(fc.selected)

# Change defaults and reset the dialog
fc.default_path = '/home/rasieira/Escritorio/OUTPUT/BATS/'

# Change hidden files
fc.show_hidden = True

# Show or hide folder icons
fc.use_dir_icons = True

# Switch to folder-only mode
fc.show_only_dirs = False

# Change the title (use '' to hide)
fc.title = '<b>Selecciona el INPUT</b>'

fc1 = FileChooser('../')

# Print the selected path, filename, or both
# if fc1.selected_path is not None:
#     print(fc1.selected_path)
# if fc1.selected_filename is not None:
#     print(fc1.selected_filename)
# if fc1.selected is not None:
#     print(fc1.selected)

# Change defaults and reset the dialog
fc1.default_path = '/home/rasieira/Escritorio/modelos/'


# Change hidden files
fc1.show_hidden = True

# Show or hide folder icons
fc1.use_dir_icons = True

# Switch to folder-only mode
fc1.show_only_dirs = False

# Change the title (use '' to hide)
fc1.title = '<b>Selecciona la carpetas con los Modelos(.bin)</b>'

In [12]:
fc2 = FileChooser('../')

# Print the selected path, filename, or both
# if fc2.selected_path is not None:
#     print(fc2.selected_path)
# if fc2.selected_filename is not None:
#     print(fc2.selected_filename)
# if fc2.selected is not None:
#     print(fc2.selected)

# Change defaults and reset the dialog
fc2.default_path = '/home/rasieira/Escritorio/OUTPUT/BATS/'
# Change hidden files
fc2.show_hidden = True

# Show or hide folder icons
fc2.use_dir_icons = True

# Switch to folder-only mode
fc2.show_only_dirs = False

# Change the title (use '' to hide)
fc2.title = '<b>Selecciona el OUTPUT de los RESULTADOS</b>'

In [13]:
btn_run = widgets.Button(description='Sacar Predicciones Ruido')
btn_run.on_click(on_click_classify)

In [14]:
fc3 = FileChooser('../')

# Print the selected path, filename, or both
# if fc2.selected_path is not None:
#     print(fc2.selected_path)
# if fc2.selected_filename is not None:
#     print(fc2.selected_filename)
# if fc2.selected is not None:
#     print(fc2.selected)

# Change defaults and reset the dialog
fc3.default_path = '/home/rasieira/Escritorio/INPUT/'
# Change hidden files
fc3.show_hidden = True

# Show or hide folder icons
fc3.use_dir_icons = True

# Switch to folder-only mode
fc3.show_only_dirs = False

# Change the title (use '' to hide)
fc3.title = '<b>Selecciona el INPUT de los AUDIOS(con ruido)</b>'
fc4 = FileChooser('../')

# Print the selected path, filename, or both
# if fc2.selected_path is not None:
#     print(fc2.selected_path)
# if fc2.selected_filename is not None:
#     print(fc2.selected_filename)
# if fc2.selected is not None:
#     print(fc2.selected)

# Change defaults and reset the dialog
fc4.default_path = '/home/rasieira/Escritorio/modelos/'
# Change hidden files
fc4.show_hidden = True

# Show or hide folder icons
fc4.use_dir_icons = True

# Switch to folder-only mode
fc4.show_only_dirs = False

# Change the title (use '' to hide)
fc4.title = '<b>Selecciona la carpetas con los Modelos(.bin)</b>'
fc5 = FileChooser('../')

# Print the selected path, filename, or both
# if fc2.selected_path is not None:
#     print(fc2.selected_path)
# if fc2.selected_filename is not None:
#     print(fc2.selected_filename)
# if fc2.selected is not None:
#     print(fc2.selected)

# Change defaults and reset the dialog
fc5.default_path = '/home/rasieira/Escritorio/'
# Change hidden files
fc5.show_hidden = True

# Show or hide folder icons
fc5.use_dir_icons = True

# Switch to folder-only mode
fc5.show_only_dirs = False

# Change the title (use '' to hide)
fc5.title = '<b>Selecciona el OUTPUT de los RESULTADOS</b>'
btn_run_2 = widgets.Button(description='Sacar Predicciones Grupos/Subgrupos')
btn_run_2.on_click(on_click_classify_2)
aparicion_noise=widgets.BoundedFloatText(
    value=0.1,
    min=0,
    max=1.0,
    description='aparicion',
    disabled=False)
umbral_ruido=widgets.BoundedFloatText(
    value=0.65,
    min=0,
    max=1.0,
    description='umbral noise',
    disabled=False)
umbral_bats=widgets.BoundedFloatText(
    value=0.95,
    min=0,
    max=1.0,
    description='umbral bats',
    disabled=False)
umbral_groups=widgets.BoundedFloatText(
    value=0.98,
    min=0,
    max=1.0,
    description='umbral conf',
    disabled=False)
aparicion_species=widgets.BoundedFloatText(
    value=0.1,
    min=0,
    max=1.0,
    description='aparicion',
    disabled=False)

In [15]:
fc6 = FileChooser('../')

# Print the selected path, filename, or both
# if fc2.selected_path is not None:
#     print(fc2.selected_path)
# if fc2.selected_filename is not None:
#     print(fc2.selected_filename)
# if fc2.selected is not None:
#     print(fc2.selected)

# Change defaults and reset the dialog
fc6.default_path = '/home/rasieira/Escritorio/OUTPUT/BATS/MULTIPLES/'
# fc6.default_path = '/home/rasieira/Escritorio/INPUT/'
# Change hidden files
fc6.show_hidden = True

# Show or hide folder icons
fc6.use_dir_icons = True

# Switch to folder-only mode
fc6.show_only_dirs = True

# Change the title (use '' to hide)
fc6.title = '<b>Selecciona el INPUT de los AUDIOS(con ruido)</b>'
fc7 = FileChooser('../')

# Print the selected path, filename, or both
# if fc2.selected_path is not None:
#     print(fc2.selected_path)
# if fc2.selected_filename is not None:
#     print(fc2.selected_filename)
# if fc2.selected is not None:
#     print(fc2.selected)

# Change defaults and reset the dialog
fc7.default_path = '/home/rasieira/Escritorio/OUTPUT/BATS/MULTIPLES/'
# fc7.default_path = '/home/rasieira/Escritorio/OUTPUT/'
# Change hidden files
fc7.show_hidden = True

# Show or hide folder icons
fc7.use_dir_icons = True

# Switch to folder-only mode
fc7.show_only_dirs = True

# Change the title (use '' to hide)
fc7.title = '<b>Selecciona el OUTPUT de los RESULTADOS</b>'
btn_run_3 = widgets.Button(description='Sacar Predicciones Grupos/Subgrupos')
btn_run_3.on_click(on_click_classify_3)

<hr>
<p><br></p>
<p><strong>Para eliminar ruido</strong></p>

In [16]:
VBox([fc3,fc4,fc5,aparicion_noise,umbral_ruido,umbral_bats,btn_run_2,out_pl2])

<hr>
<p><br></p>
<p><strong>Para clasificar grupos fonicos y el subgrupo PPIT</strong></p>

In [17]:
VBox([fc,fc1,fc2,aparicion_species,umbral_groups,btn_run,out_pl])

<hr>
<p><br></p>
<p><strong>Para generar los spectogramas</strong></p>

In [18]:
VBox([fc6,fc7,btn_run_3,out_pl3])